In [31]:
from dotenv import load_dotenv
import os

## [Overview](markdown/overview.md)

### [Simple RAG query](markdown/simple-rag-query.md)

In [32]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings, ChatOllama

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embedding_function = OllamaEmbeddings(model='mxbai-embed-large')

current_dir = os.getcwd() 
persistent_directory = os.path.join(current_dir, "db", "chroma_db")

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embedding_function, persist_directory=persistent_directory)

vectorstore.persist()


retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
#llm = ChatOllama(model="qwen2.5:32b-instruct-q4_K_M")
llm = ChatOllama(model="llama3.1")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique that breaks down complex tasks into smaller, manageable steps by instructing the model to "think step by step". This helps to decompose hard tasks into simpler ones, enhancing model performance on complex tasks. It transforms big tasks into multiple manageable tasks and provides insight into the model\'s thinking process.'

## Second Example
### In this instance we are using an already indexed vector db
### The 'chroma_db' persisted db contains chunks from a text file of the odyssey 
### We ask Who is Odysseus' wife?

In [33]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings, ChatOllama

embedding_function = OllamaEmbeddings(model='mxbai-embed-large')

# Get the current directory
current_dir = os.getcwd()

# Get the directory above the current directory
parent_dir = os.path.dirname(current_dir)

persistent_directory = os.path.join(parent_dir, "db", "chroma_db")

db = Chroma(persist_directory=persistent_directory,embedding_function=embedding_function)

bookretriever = db.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOllama(model="llama3.1")

# Chain
rag_chain = (
    # {"context": retriever | format_docs, "question": RunnablePassthrough()}
    {"context": bookretriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("Who is Odysseus' wife?")

'Penelope is Odysseus\' wife. She is described as a "very admirable woman" and has an "excellent nature". When Agamemnon speaks of her, he notes that she would not murder her husband like his own wife did to him.'

## Part 3: Retrieval
In the following code sample we can restrict the number of documents returned by setting the {"k": 1} arguement - in this case just one

In [34]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

docs = retriever.get_relevant_documents("What is Task Decomposition?")

len(docs)

1

## Generation
### Simple example without a retriever 
A Chat prompt is given and piped into the llm with the previosly created docs

In [35]:
from langchain.prompts import ChatPromptTemplate
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})]


In [36]:
# LLM
llm = ChatOllama(model="llama3.1")

In [37]:
# Chain
chain = prompt | llm

In [38]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='Task Decomposition is a technique used in LLM-powered autonomous agent systems where complicated tasks are broken down into smaller, more manageable steps to be planned ahead. It involves instructing the model to "think step by step" utilizing test-time computation to decompose hard tasks.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-01-26T21:25:50.164495261Z', 'done': True, 'done_reason': 'stop', 'total_duration': 863361775, 'load_duration': 33565453, 'prompt_eval_count': 185, 'prompt_eval_duration': 22000000, 'eval_count': 55, 'eval_duration': 806000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6217b1b2-2058-455d-93d0-14049eae6baf-0', usage_metadata={'input_tokens': 185, 'output_tokens': 55, 'total_tokens': 240})

### Using a prompt from langchain hub

In [39]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [40]:
print(prompt_hub_rag)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a component within a LLM-powered autonomous agent system that involves breaking down complicated tasks into smaller, more manageable steps, utilizing techniques such as Chain of Thought (CoT) to enhance model performance on complex tasks.'

## [Query Transformations](markdown/query-transformations.md)

## [Part 5: Multi Query](markdown/multi-query.md)

In [42]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen2.5:32b-instruct-q4_K_M")

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)


In [43]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

1

In [44]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(model="qwen2.5:32b-instruct-q4_K_M")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition in the context of Large Language Model (LLM) agents involves breaking down a complicated task into smaller, more manageable steps. This method leverages a technique known as Chain of Thought (CoT), where the model is prompted to think through the problem step by step. By doing so, it can handle complex tasks more effectively by utilizing its computational resources over multiple simpler sub-tasks rather than trying to solve the entire task at once. This approach not only enhances the performance of the LLM on complex tasks but also provides insights into how the model processes and thinks through these problems.'

## Second multi query example

In [45]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.1")

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "How did Odysseus get home?"
retrieval_chain = generate_queries | bookretriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(model="llama3.1")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


'The passage doesn\'t explicitly state how Odysseus got home. However, it does mention that "Ulysses woke up once more upon his own soil", suggesting that he was transported back to Ithaca somehow.\n\nTo answer your question based on the context of Homer\'s Odyssey (which this text appears to be summarizing), Odysseus gets home with the help of the gods, particularly Athena and Poseidon. After many years away fighting in the Trojan War, Odysseus faces numerous challenges and adventures as he tries to return to Ithaca. He is aided by the goddess Athena, who helps him navigate his journey and ultimately reclaim his throne.\n\nIn this specific passage (Document id \'c397882f-16a3-436e-ac08-918635ba7fc1\'), it mentions that "Jove\'s daughter Minerva had made it a foggy day, so that people might not know of his having come", which suggests that Odysseus\' return is facilitated by the goddess Athena (also known as Minerva).'

## [Part 6: RAG-Fusion](markdown/rank-fusion.md)

In [46]:
# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)
question = "What is task decomposition for LLM agents?"

In [47]:
from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [48]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

2

In [49]:

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition involves breaking down large tasks into smaller, manageable subgoals to enable efficient handling of complex tasks. This process allows the agent to "think step by step" and utilize more test-time computation to decompose hard tasks into simpler steps. Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to think in this manner, transforming big tasks into multiple manageable tasks and providing an interpretation of the model\'s thinking process.'

### Second Rank Fusion Example

In [50]:
question = "Who was the wife of Odysseus?"
retrieval_chain_rag_fusion = generate_queries | bookretriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Penelope.'

## [Part 7: Decomposition](markdown/decomposition.md)

In [51]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [52]:
llm = ChatOllama(model="llama3.1")
# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [53]:
print(questions)

['Here are three sub-queries related to the input question:', '', '1. **What is the role of Natural Language Processing (NLP) in LLM-powered autonomous agents?**', '\t* This query breaks down the input into a more specific question about one of the key components involved in building an LLM-powered autonomous agent system.', '2. **How do Large Language Models (LLMs) interact with other AI/ML technologies to enable autonomy?**', '\t* This sub-question focuses on the interaction between LLMs and other technologies, such as computer vision or reinforcement learning, that are necessary for an autonomous agent to function.', '3. **What is the relationship between cognitive architectures and LLM-powered autonomous agents in terms of decision-making and action selection?**', '\t* This query delves into the high-level architecture and decision-making processes involved in building an autonomous agent powered by an LLM.', '', 'These sub-queries can be answered independently, providing a more de

Answer recursively

In [54]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [55]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

llm = ChatOllama(model="llama3.1")
q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [56]:
print(answer)

Based on the provided context and background information, I will attempt to answer the question.


The high-level architecture and decision-making processes involved in building an autonomous agent powered by an LLM (Large Language Model) can be inferred as follows:


**Key Components**:


1.  **Planning**: This involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
2.  **Memory**: This enables agents to store and retrieve information, facilitating learning and decision-making.

**LLM's Role**: The LLM functions as the agent’s brain, playing a central role in decision-making and action selection through its planning and reflection/refinement mechanisms.


**Relationship with Cognitive Architectures**: While cognitive architectures are not explicitly mentioned in the provided context, their functions can be inferred from the description of the planning and memory components. This suggests that LLM-powered autonomous agents sha

### second decomposition example

In [57]:
# Run
question = "What happens during Odysseus's encounter with Polyphemus?"
questions = generate_queries_decomposition.invoke({"question":question})

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | bookretriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [58]:
print(answer)

Based on the provided context and background knowledge, here's an answer to the question:

The decision of Odysseus to blind the Cyclops has multiple symbolic meanings that reflect the themes and motifs present in Homer's Odyssey. On one level, the blinding serves as a form of revenge for the Cyclops' brutal treatment of Odysseus and his men. The act can be seen as a manifestation of Odysseus's desire for justice and retribution against those who have wronged him.

On another level, the blinding can be interpreted symbolically as a representation of Odysseus's own journey towards enlightenment and self-discovery. By blinding the Cyclops, Odysseus is able to break free from the monster's control and assert his own identity and agency. This act can be seen as a metaphor for Odysseus's struggle to navigate the challenges and obstacles that lie ahead on his journey home.

Furthermore, the blinding of the Cyclops serves as a warning to other characters in the Odyssey about the consequences 

## [Part 8: Step Back](markdown/stepback.md)

In [59]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [61]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How do large language model agents break down complex tasks into more manageable parts?'

In [62]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition in the context of Large Language Model (LLM) powered autonomous agent systems refers to the process of breaking down complex tasks into smaller, more manageable sub-tasks. This is a crucial step in enabling LLM agents to plan and execute complex tasks efficiently.\n\nThe concept of task decomposition is closely related to the "Chain of Thought" (CoT) prompting technique, which involves instructing the model to think step-by-step and utilize more test-time computation to decompose hard tasks into smaller steps. This approach allows the model to provide a clear interpretation of its thinking process and can be particularly effective for complex tasks that require multiple steps.\n\nIn the context of HuggingGPT (Shen et al., 2023), task decomposition is one of the four stages in the system\'s workflow, along with task planning, execution, and monitoring. During task planning, LLMs are used to parse user requests into multiple tasks, each associated with attributes such

### second example of step back query transformation

In [63]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "In the Odyssey, who is Telemachus?"
generate_queries_step_back.invoke({"question": question})

"who are the main characters in Homer's epic poem?"

In [64]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | bookretriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | bookretriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'According to the text, Telemachus is a character in Homer\'s epic poem "The Odyssey". He is the son of King Odysseus and Queen Penelope. During his father\'s prolonged absence, Telemachus tries to maintain order in their kingdom and defend his mother against the suitors who are vying for her hand in marriage.\n\nIn Book II of the Odyssey, Telemachus attends a council at Pylos where he meets with King Nestor, who is an old friend of Odysseus. Through this meeting, Telemachus gains information about his father\'s whereabouts and begins to understand that he must take action to reclaim his kingdom.\n\nThroughout the poem, Telemachus grows from a young prince into a brave and determined leader who plays a crucial role in the ultimate downfall of the suitors and the return of Odysseus to Ithaca.'